In [97]:
xls.prep <- function(formula,data,dependent_var){
    
    matrix.baked <- stats::model.matrix.lm(formula,data,na.action = 'na.pass')
    
    df.baked <- base::as.data.frame(matrix.baked)
    
    base::rm(matrix.baked)
    
    df.baked[[dependent_var]] <- data[[dependent_var]]
    
    if(base::anyNA(df.baked)){

        count_na <- base::nrow(df.baked[!stats::complete.cases(df.baked),])
        msg_format <- '%s rows with missing data were dropped.'
        base::message(base::sprintf(msg_format,count_na))

        df.baked <- df.baked[stats::complete.cases(df.baked),]

    }

    dependent_var_vector <- df.baked[[dependent_var]]

    df.baked[[dependent_var]] <- NULL
        
    varnum <- base::ncol(df.baked)
    
    varnames <- base::colnames(df.baked)
    
    coefnames <- base::paste0('x[',1:varnum,']')
    
    regformat <- base::paste(base::rep('%s*%s',varnum),collapse = ' + ')
    
    coef_var_match <- NULL
    
    for(i in 1:varnum){
        coef_var_match[i] <- base::sprintf('"%s",df.baked[["%s"]]',coefnames[i],varnames[i])
    }
    
    coef_var_match <- base::paste(coef_var_match,collapse = ',')
    
    symbolic_error_calculation <- base::sprintf('base::sprintf("%s",%s)',regformat,coef_var_match)
    
    base::eval(base::parse(text = base::sprintf('df.baked[["error_symbolic"]] <- %s',symbolic_error_calculation)))
    
    df.baked[["error_symbolic"]] <- base::sprintf('(%s - %s)^2',df.baked[["error_symbolic"]],dependent_var_vector)

    base::list(data=df.baked,independent_var=varnames)

}

In [98]:
xls.objfun <- function(data,error_column_name,error_weights,error_ahead_level){
    
    df <- data
    
    sample_size <- nrow(df)
    
    df$ahead_error_symbolic <- base::sapply(
      1:sample_size,
      function(x) base::paste(df[[error_column_name]][x:min(x+error_ahead_level-1, sample_size)], collapse = " add ")
    )
    
    df$ahead_num <- sapply(strsplit(df$ahead_error_symbolic,split = ' add '),base::length)
    
    df <- df[df[['ahead_num']] == error_ahead_level,]
    
    df$ahead_error_symbolic <- base::strsplit(df$ahead_error_symbolic,' add ')
    
    new_sample_size <- base::nrow(df)
    
    df$error_weights <- base::rep(base::list(error_weights),new_sample_size)
    
    new_error_column <- base::paste0(error_column_name,'_weighted')
    
    df[[new_error_column]] <- base::mapply(
      base::paste,
      df$error_weights,
      df$ahead_error_symbolic,
      sep = "*",
      collapse = " + "
    )

    sum_of_errors <- base::paste(df[[new_error_column]],collapse = ' + ')
    
    multivariate_sse <- base::gsub("x\\[(\\d)\\]", "x\\1", sum_of_errors)
    
    base::suppressMessages({polynomial_sse <-  mpoly::mp(multivariate_sse, stars = TRUE)})
    
    base::suppressMessages({objfun <-  base::as.function(polynomial_sse, varorder = base::sort(mpoly::vars(polynomial_sse)))})
    
    base::return(objfun)
}

In [99]:
xls.objfun <- function(data,error_column_name,error_weights,error_ahead_level){
    
    df <- data
    
    sample_size <- nrow(df)
    
    df$ahead_error_symbolic <- base::sapply(
      1:sample_size,
      function(x) base::paste(df[[error_column_name]][x:min(x+error_ahead_level-1, sample_size)], collapse = " add ")
    )
    
    df$ahead_num <- sapply(strsplit(df$ahead_error_symbolic,split = ' add '),base::length)
    
    df <- df[df[['ahead_num']] == error_ahead_level,]
    
    df$ahead_error_symbolic <- base::strsplit(df$ahead_error_symbolic,' add ')
    
    new_sample_size <- base::nrow(df)
    
    df$error_weights <- base::rep(base::list(error_weights),new_sample_size)
    
    new_error_column <- base::paste0(error_column_name,'_weighted')
    
    df[[new_error_column]] <- base::mapply(
      base::paste,
      df$error_weights,
      df$ahead_error_symbolic,
      sep = "*",
      collapse = " + "
    )

    sum_of_errors <- base::paste(df[[new_error_column]],collapse = ' + ')
    
    multivariate_sse <- base::gsub("x\\[(\\d)\\]", "x\\1", sum_of_errors)
    
    base::suppressMessages({polynomial_sse <-  mpoly::mp(multivariate_sse, stars = TRUE)})
    
    base::suppressMessages({objfun <-  base::as.function(polynomial_sse, varorder = base::sort(mpoly::vars(polynomial_sse)))})
    
    base::return(objfun)
}

In [100]:
df <- datasets::airquality
ordered_df <- df[with(df,order(Month,Day)),]
model <- xls.fit(Ozone ~ Solar.R + Wind + Temp,ordered_df,error_ahead_level = 4)

42 rows with missing data were dropped.



In [101]:
df <- datasets::airquality
ordered_df <- df[with(df,order(Month,Day)),]
model <- xls.fit(Ozone ~ Solar.R + Wind + Temp,ordered_df,error_ahead_level = 4)

42 rows with missing data were dropped.



In [102]:
model2 <- lm(Ozone ~ Solar.R + Wind + Temp,ordered_df)

In [103]:
model


Call:
xls.fit(formula = Ozone ~ Solar.R + Wind + Temp, data = ordered_df, 
    error_weights = c(0.4, 0.3, 0.2, 0.1), error_ahead_level = 4)

Coefficients:
(Intercept)      Solar.R         Wind         Temp  
  -63.25402      0.05742     -3.34202      1.64686  


In [104]:
model2


Call:
lm(formula = Ozone ~ Solar.R + Wind + Temp, data = ordered_df)

Coefficients:
(Intercept)      Solar.R         Wind         Temp  
  -64.34208      0.05982     -3.33359      1.65209  
